In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
from pathlib import Path
import yaml

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../source")
from helper import init_model
from datasets.ptz_dataset import PTZImageDataset

In [ ]:
with open("../configs/Config_file.yaml", 'r') as y_file:
    args = yaml.load(y_file, Loader=yaml.FullLoader)
use_bfloat16 = args['meta']['use_bfloat16']
model_name = args['meta']['model_name']
pred_depth = args['meta']['pred_depth']
pred_emb_dim = args['meta']['pred_emb_dim']
patch_size = args['mask']['patch_size']  # patch-size for model training
crop_size = args['data']['crop_size']
batch_size = args['data']['batch_size']
camera_brand = args['meta']['camera_brand']


In [ ]:
encoder, predictor = init_model(
        device="cpu",
        patch_size=patch_size,
        crop_size=crop_size,
        pred_depth=pred_depth,
        pred_emb_dim=pred_emb_dim,
        model_name=model_name)

In [ ]:
import importlib
import source
importlib.reload(source)
from source.run_jepa import get_position_from_label, forward_context, arrage_inputs

In [ ]:
from source.datasets.ptz_dataset import PTZImageDataset

In [ ]:
from torchvision.transforms import ToTensor

In [ ]:
from source.transforms import make_transforms

In [ ]:
data = PTZImageDataset('../../collected_imgs_bu2', transform=make_transforms())
dataloader = DataLoader(data, batch_size=batch_size, shuffle=False)
ipe = len(dataloader)

In [ ]:
img_lab = next(iter(dataloader))

In [ ]:
isinstance(img_lab[1], Iterable)

In [ ]:
pos = get_position_from_label(img_lab[1])

In [ ]:
context_imgs, context_poss, target_imgs, target_poss = arrage_inputs(img_lab[0], pos, "cpu")

In [ ]:
z, r = forward_context(context_imgs, context_poss, target_poss, encoder, predictor, camera_brand)

In [ ]:
z.shape